# Лекция 1. Задание 1

Программная реализация регулярной расчетной сетки, заданной массивом узлов $x_i$, $y_j$, $z_k$. Необходимо определить функции для вычисления центров ячеек и размеров ячеек. Учесть наличие фиктивных ячеек для задания граничных условий.

![](./assets/task01.png)

## Решение

In [1]:
from itertools import product

import numpy as np

In [2]:
class Cell:
    def __init__(self, nodes: list[float] = np.zeros((3, 2))):
        nodes = np.array(nodes)
        assert (nodes.shape == (1, 2) or
                nodes.shape == (2, 2) or
                nodes.shape == (3, 2))
        self._nodes = nodes


    @property
    def nodes(self) -> list[float]:
        return self._nodes


    def center(self) -> float:
        return tuple(
            (self.nodes[i, 0] + self.nodes[i, 1]) / 2
            for i in range(self.nodes.shape[0]))


    def size(self) -> tuple[float]:
        return tuple(
            (self.nodes[i, 1] - self.nodes[i, 0])
            for i in range(self.nodes.shape[0]))


    def volume(self) -> float:
        return np.prod(self.size())

In [3]:
class Grid:
    def __init__(self, nodes: list[list[float]], ghostlen: int = 2):
        assert len(nodes) == 1 or len(nodes) == 2 or len(nodes) == 3
        self._nodes = nodes
        assert ghostlen >= 0
        self._ghostlen = ghostlen

        self._ilen = tuple(len(dim) + 2 * self.ghostlen for dim in self.nodes)

        self._imax = tuple(self.ilen[i] - self.ghostlen for i in range(len(self.nodes)))

        self._imin = tuple(self.ghostlen for _ in range(len(self.nodes)))

        # Fill volume with Cell()
        self._cells = np.zeros(
            [ilen - 1 for ilen in self.ilen]).tolist()
        set_ghosts = lambda cells: (
            list(map(lambda l: set_ghosts(l), cells))
            if isinstance(cells, list)
            else list(map(lambda _: Cell(), [None]))[0])
        self._cells = set_ghosts(self._cells)

        # Fill volume with Cell([x_i, y_j, z_k]) excluding ghosts
        for idxs in product(*[list(range(node.size - 1)) for node in self.nodes]):
            get_shifted_idx = lambda idx: idxs[idx] + self.imin[idx]

            cell = Cell([self.nodes[idx][idxs[idx] : idxs[idx] + 2] for idx in range(len(self.nodes))])
            match len(self.nodes):
                case 1:
                    self.cells[get_shifted_idx(0)] = cell
                case 2:
                    self.cells[get_shifted_idx(0)][get_shifted_idx(1)] = cell
                case 3:
                    self.cells[get_shifted_idx(0)][get_shifted_idx(1)][get_shifted_idx(2)] = cell


    @property
    def cells(self) -> list[list[list[Cell]]]:
        return self._cells


    @property
    def ghostlen(self) -> int:
        return self._ghostlen


    @property
    def ilen(self) -> int:
        return self._ilen


    @property
    def imax(self) -> int:
        return self._imax


    @property
    def imin(self) -> int:
        return self._imin


    @property
    def nodes(self) -> list[float]:
        return self._nodes

In [9]:
g3 = Grid([np.linspace(-3, 3, 7), np.linspace(-1, 1, 3), np.linspace(-2, 2, 5)], ghostlen=0)

In [10]:
outstr = ""
for z in g3.cells:
    for y in z:
        for x in y:
            outstr += f"{x.volume()}\t\t"
        outstr += "\n"
    outstr += "\n\n"
print(f"{outstr}\n")

1.0		1.0		1.0		1.0		
1.0		1.0		1.0		1.0		


1.0		1.0		1.0		1.0		
1.0		1.0		1.0		1.0		


1.0		1.0		1.0		1.0		
1.0		1.0		1.0		1.0		


1.0		1.0		1.0		1.0		
1.0		1.0		1.0		1.0		


1.0		1.0		1.0		1.0		
1.0		1.0		1.0		1.0		


1.0		1.0		1.0		1.0		
1.0		1.0		1.0		1.0		




